In [1]:
from pkg_resources import resource_filename
import numpy as np
import os
import copy 

import simtk.openmm.app as app
import simtk.openmm as openmm
import simtk.unit as unit

from perses.utils.smallmolecules import render_protein_residue_atom_mapping
from perses.rjmc import topology_proposal
from perses.rjmc.topology_proposal import append_topology
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.atom_mapping import AtomMapper, InvalidMappingException, AtomMapping

from openeye import oechem, oedepict
from openmmforcefields.generators import SystemGenerator
from openmmtools.constants import kB

temperature = 300*unit.kelvin
# Compute kT and inverse temperature.
kT = kB * temperature
beta = 1.0 / kT
ENERGY_THRESHOLD = 1e-6
PROHIBITED_RESIDUES = ['CYS']

INFO:rdkit:Enabling RDKit 2021.09.4 jupyter extensions
INFO:numexpr.utils:Note: detected 80 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 80 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
def generate_old_top_pos_sys(res_name):
    """
    Generate old topology, positions, system, and system generator
    
    Parameters
    ----------
    res_name : str
        three letter amino acid name of the residue for which to generate the topology, positions, system
        
    Returns
    -------
    topology
    positions
    system
    system_generator
    
    """
    
    # Load PDB of dipeptide, generated by using the geometry engine (with ALA as the old positions)
    pdb = app.PDBFile(f"../../input/{res_name.lower()}_vacuum.pdb")
    topology = pdb.topology
    positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in pdb.positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
    
    # Create a system generator
    system_generator = SystemGenerator(['amber14/protein.ff14SB.xml'],
                                   barostat=None,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                        'ewaldErrorTolerance': 1e-4,
                                                        'constraints' : app.HBonds,
                                                        'hydrogenMass' : 4 * unit.amus},
                                    nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                    small_molecule_forcefield='gaff-2.11',
                                    molecules=None,
                                    cache=None)
    
    # Create a system
    system = system_generator.create_system(topology) # Update the parametrization scheme to amberff14sb

    return topology, positions, system, system_generator

def generate_inputs(new_res_name, topology, system, system_generator):
    """
    Generate inputs for construct atom map: old_res, new_res

    Parameters
    ----------
    new_res_name : str
        three letter amino acid name of the residue to mutate to

    Returns
    -------
    old_res
    new_res
    """
    
    new_res_name = new_res_name.upper()
    
    # Create a point mutation engine
    point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                                system_generator=system_generator,
                                                chain_id='A', # Denote the chain id allowed to mutate (it's always a string variable)
                                                max_point_mutants=1,
                                                residues_allowed_to_mutate=['2'], # The residue ids allowed to mutate
                                                allowed_mutations=[('2', new_res_name)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                aggregate=True) # Always allow aggregation

    # Create new objects for old and new topologies
    old_topology = app.Topology()
    append_topology(old_topology, topology)

    new_topology = app.Topology()
    append_topology(new_topology, topology)

    # Check that old_topology and old_system have same number of atoms.
    old_system = system
    old_topology_natoms = old_topology.getNumAtoms()  # number of topology atoms
    old_system_natoms = old_system.getNumParticles()
    if old_topology_natoms != old_system_natoms:
        msg = 'PolymerProposalEngine: old_topology has %d atoms, while old_system has %d atoms' % (old_topology_natoms, old_system_natoms)
        raise Exception(msg)

    # Generate old chemical state key
    old_chemical_state_key = point_mutation_engine.compute_state_key(old_topology)

    # Create residue_map
    metadata = dict()
    index_to_new_residues, metadata = point_mutation_engine._choose_mutant(old_topology, metadata)
    residue_map = point_mutation_engine._generate_residue_map(old_topology, index_to_new_residues)
    for (res, new_name) in residue_map:
        if res.name == new_name:
            # remove the index_to_new_residues entries where the topology is already mutated
            del(index_to_new_residues[res.index])

    # Add modified_aa property to residues in old topology
    for res in old_topology.residues():
        res.modified_aa = True if res.index in index_to_new_residues.keys() else False

    # Identify differences between old topology and proposed changes
    # excess_atoms : list(simtk.openmm.app.topology.Atom) atoms from existing residue not in new residue
    # excess_bonds : list(tuple (simtk.openmm.app.topology.Atom, simtk.openmm.app.topology.Atom)) bonds from existing residue not in new residue
    # missing_bonds : list(tuple (simtk.openmm.app.topology._TemplateAtomData, simtk.openmm.app.topology._TemplateAtomData)) bonds from new residue not in existing residue
    excess_atoms, excess_bonds, missing_atoms, missing_bonds = point_mutation_engine._identify_differences(old_topology, residue_map)

    # Delete excess atoms and bonds from old topology
    excess_atoms_bonds = excess_atoms + excess_bonds
    new_topology = point_mutation_engine._delete_atoms(old_topology, excess_atoms_bonds)

    # Add missing atoms and bonds to new topology
    new_topology = point_mutation_engine._add_new_atoms(new_topology, missing_atoms, missing_bonds, residue_map)

    # Generate map of old to new residue objects
    old_to_new_residues = {}
    new_residues = [residue for residue in new_topology.residues()] # Assumes all residue indices start from 0 and are contiguous
    for old_residue in old_topology.residues():
        old_to_new_residues[old_residue] = new_residues[old_residue.index]

    old_res = residue_map[0][0]
    new_res = old_to_new_residues[old_res]

    return old_res, new_res

def render_image(molecule1, molecule2, new_to_old_atom_map, filename=None):
    from openeye import oechem, oedepict
    width = 1200
    height = 600
    format = 'png'

    # Handle format
    if format is None:
        if filename is not None:
            format = oechem.OEGetFileExtension(filename)
        else:
            format = 'png'
    if not oedepict.OEIsRegisteredImageFile(format):
        raise ValueError(f'Unknown image type {format}')

    oechem.OEGenerate2DCoordinates(molecule1)
    oechem.OEGenerate2DCoordinates(molecule2)

    # Add both to an OEGraphMol reaction
    rmol = oechem.OEGraphMol()
    rmol.SetRxn(True)
    if filename is not None:
        rmol.SetTitle(filename[:7])
    def add_molecule(mol):
        # Add atoms
        new_atoms = list()
        old_to_new_atoms = dict()
        for old_atom in mol.GetAtoms():
            new_atom = rmol.NewAtom(old_atom.GetAtomicNum())
            new_atom.SetFormalCharge(old_atom.GetFormalCharge())
            new_atom.SetName(old_atom.GetName())
            new_atoms.append(new_atom)
            old_to_new_atoms[old_atom] = new_atom
        # Add bonds
        for old_bond in mol.GetBonds():
            rmol.NewBond(old_to_new_atoms[old_bond.GetBgn()], old_to_new_atoms[old_bond.GetEnd()], old_bond.GetOrder())
        return new_atoms, old_to_new_atoms

    [new_atoms_1, old_to_new_atoms_1] = add_molecule(molecule1)
    [new_atoms_2, old_to_new_atoms_2] = add_molecule(molecule2)

    # Label reactant and product
    for atom in new_atoms_1:
        atom.SetRxnRole(oechem.OERxnRole_Reactant)
    for atom in new_atoms_2:
        atom.SetRxnRole(oechem.OERxnRole_Product)

    core1 = oechem.OEAtomBondSet()
    core2 = oechem.OEAtomBondSet()
    # add all atoms to the set
    core1.AddAtoms(new_atoms_1)
    core2.AddAtoms(new_atoms_2)
    # Label mapped atoms
    core_change = oechem.OEAtomBondSet()
    index =1
    for (index2, index1) in new_to_old_atom_map.items():
        new_atoms_1[index1].SetMapIdx(index)
        new_atoms_2[index2].SetMapIdx(index)
        # now remove the atoms that are core, so only uniques are highlighted
        core1.RemoveAtom(new_atoms_1[index1])
        core2.RemoveAtom(new_atoms_2[index2])
        if new_atoms_1[index1].GetAtomicNum() != new_atoms_2[index2].GetAtomicNum():
            # this means the element type is changing
            core_change.AddAtom(new_atoms_1[index1])
            core_change.AddAtom(new_atoms_2[index2])
        index += 1
    # Set up image options
    itf = oechem.OEInterface()
    oedepict.OEConfigureImageOptions(itf)

    # Setup depiction options
    oedepict.OEConfigure2DMolDisplayOptions(itf, oedepict.OE2DMolDisplaySetup_AromaticStyle)
    opts = oedepict.OE2DMolDisplayOptions(width, height, oedepict.OEScale_AutoScale)
    oedepict.OESetup2DMolDisplayOptions(opts, itf)
    opts.SetBondWidthScaling(True)
#     opts.SetAtomPropertyFunctor(oedepict.OEDisplayAtomMapIdx())
    opts.SetAtomColorStyle(oedepict.OEAtomColorStyle_WhiteMonochrome)
    
    class LabelAtomNames(oedepict.OEDisplayAtomPropBase):
        def __init__(self):
            oedepict.OEDisplayAtomPropBase.__init__(self)

        def __call__(self, atom):
            if atom.GetMapIdx() != 0:
                return f"{atom.GetMapIdx()} ({atom.GetName()})"
            else: 
                return f"{atom.GetName()}"
    
        def CreateCopy(self):
            # __disown__ is required to allow C++ to take
            # ownership of this object and its memory
            copy = LabelAtomNames()
            return copy.__disown__()
    atomlabel = LabelAtomNames()
    opts.SetAtomPropertyFunctor(atomlabel)
    opts.SetAtomPropLabelFontScale(0.6)
    opts.SetAtomPropLabelFont(oedepict.OEFont(oechem.OEBlue))
    opts.SetTitleFontScale(0.5)

    # Depict reaction with component highlights
    oechem.OEGenerate2DCoordinates(rmol)
    display = oedepict.OE2DMolDisplay(rmol, opts)

    if core1.NumAtoms() != 0:
        oedepict.OEAddHighlighting(display, oechem.OEColor(oechem.OEPink),oedepict.OEHighlightStyle_Stick, core1)
    if core2.NumAtoms() != 0:
        oedepict.OEAddHighlighting(display, oechem.OEColor(oechem.OEPurple),oedepict.OEHighlightStyle_Stick, core2)
    if core_change.NumAtoms() != 0:
        oedepict.OEAddHighlighting(display, oechem.OEColor(oechem.OEGreen),oedepict.OEHighlightStyle_Stick, core_change)

    if filename is not None:
        ofs = oechem.oeofstream()
        if not ofs.open(filename):
            raise Exception('Cannot open output file %s' % filename)
        oedepict.OERenderMolecule(ofs, format, display)
        ofs.close()
    else:
        from IPython.display import Image
        oeimage = oedepict.OEImage(width, height)
        oedepict.OERenderMolecule(oeimage, display)
        string = oedepict.OEWriteImageToString(format, oeimage)
        return Image(string)
    
def construct_atom_map(old_res, new_res, map_strength=0):
    """
    Generate atom map
    
    Parameters
    ----------
    old_res : 
    new_res : 
        
    
    Returns
    -------
    
    """
    
    # Generate oemols
    current_residue_pdb_filename = resource_filename('perses', os.path.join('data', 'amino_acid_templates', f"{old_res.name}.pdb"))
    proposed_residue_pdb_filename = resource_filename('perses', os.path.join('data', 'amino_acid_templates', f"{new_res.name}.pdb"))
    current_oemol = PointMutationEngine.generate_oemol_from_pdb_template(current_residue_pdb_filename)
    proposed_oemol = PointMutationEngine.generate_oemol_from_pdb_template(proposed_residue_pdb_filename)

    # Set pattern and target (which are the variables openeye uses in example mapping code)
    pattern = current_oemol
    target = proposed_oemol
    
    # Set atom and bond expressions
    bondexpr = oechem.OEExprOpts_BondOrder | oechem.OEExprOpts_EqSingleDouble
    if map_strength == 0:
        atomexpr = oechem.OEExprOpts_Valence
    elif map_strength == 1:
        atomexpr = oechem.OEExprOpts_Valence | oechem.OEExprOpts_Hybridization
    elif map_strength == 2:
        atomexpr = oechem.OEExprOpts_Valence | oechem.OEExprOpts_Hybridization | oechem.OEExprOpts_Aromaticity
    elif map_strength == 3:
        atomexpr = oechem.OEExprOpts_Valence | oechem.OEExprOpts_Hybridization | oechem.OEExprOpts_Aromaticity | oechem.OEExprOpts_AtomicNumber
        
    # Create maximum common substructure object
    mcss = oechem.OEMCSSearch(pattern, atomexpr, bondexpr, oechem.OEMCSType_Approximate)
    
    # Always map backbone atoms
    if old_res.name == 'GLY' or new_res.name == 'GLY':
        backbone_atoms = ["N", "H", "C", "O", "CA", "H'"] # Do not map GLY HAs
    else:
        backbone_atoms = ["N", "H", "C", "O", "CA", "HA", "H'"] 
    d_old_backbone_atoms = {atom.GetName() : atom for atom in mcss.GetPattern().GetAtoms() if atom.GetName() in backbone_atoms} # Need to use mcss.GetPattern() instead of pattern here, otherwise AddConstraint will return false
    d_new_backbone_atoms = {atom.GetName() : atom for atom in target.GetAtoms() if atom.GetName() in backbone_atoms}
    assert len(d_old_backbone_atoms) == len(d_new_backbone_atoms)
    for backbone_atom in d_old_backbone_atoms:
        assert mcss.AddConstraint(oechem.OEMatchPairAtom(d_old_backbone_atoms[backbone_atom], d_new_backbone_atoms[backbone_atom]))
    
    # Set scoring function
    mcss.SetMCSFunc(oechem.OEMCSMaxAtomsCompleteCycles())
    
    # Loop over matches
    atom_mappings = []
    unique = True # When doing unique matching, two subgraph matches which cover the same atoms, albeit in different orders, will be called duplicates and it will be discarded.
    for i, match in enumerate(mcss.Match(target, unique)):
        atom_mapping = {}
        for match_atom in match.GetAtoms():
            atom_mapping[match_atom.target.GetIdx()] = match_atom.pattern.GetIdx()
        atom_mappings.append(atom_mapping)
    print(f"{old_res.name}->{new_res.name} Number of maps: {len(atom_mappings)}")
    
    # Return highest scoring mapping (with most atoms mapped), if there is a tie, return the first occurrence
    scores = np.array([ len(atom_mapping) for atom_mapping in atom_mappings ])
    best_map_index = np.argmax(scores)
    atom_mapping_oemol = atom_mappings[best_map_index]
    
    # Remove H', as this is not present in the openmm residue
    new_index_to_remove = [atom.GetIdx() for atom in proposed_oemol.GetAtoms() if atom.GetName() == "H'"][0]
    del atom_mapping_oemol[new_index_to_remove]
    
    # Convert mapping to use openmm indices instead of oemol indices
    old_oemol_to_openmm_map = {current_oemol.GetAtom(oechem.OEHasAtomName(atom.name)).GetIdx(): atom.index  for atom in old_res.atoms()}
    new_oemol_to_openmm_map = {proposed_oemol.GetAtom(oechem.OEHasAtomName(atom.name)).GetIdx(): atom.index for atom in new_res.atoms()}
    atom_mapping_openmm = [(new_oemol_to_openmm_map[new_idx], old_oemol_to_openmm_map[old_idx]) for new_idx, old_idx in atom_mapping_oemol.items()]
    print(atom_mapping_openmm)
    
    # Convert mapping to use atom names
    new_index_to_name = {atom.index: atom.name for atom in new_res.atoms()}
    old_index_to_name = {atom.index: atom.name for atom in old_res.atoms()}
    name_map = [(new_index_to_name[new_idx], old_index_to_name[old_idx]) for new_idx, old_idx in atom_mapping_openmm]
    print(name_map)
    
    return current_oemol, proposed_oemol, atom_mapping


In [5]:
# amino_acids = ['ALA', 'ARG', 'ASH', 'ASN', 'ASP', 'CYS', 'GLN', 'GLH', 'GLU', 'GLY', 
#                'HID', 'HIE', 'HIP', 'HIS', 'ILE', 'LEU', 'LYN',  'LYS',  'MET', 'PHE',
#                     'SER', 'THR', 'TRP', 'TYR', 'VAL']
amino_acids = ['ARG', 'HIS', 'LYS', 'ASP', 'GLU', 
               'SER', 'THR', 'ASN', 'GLN', 'CYS', 'GLY', 
               'ALA', 'VAL', 'ILE', 'LEU', 'MET',  'PHE',  'TYR', 'TRP']
old_res_name = "glu"
amino_acids.remove(old_res_name.upper())

i = 0
for map_strength in range(4):
    for aa in amino_acids:
        new_res_name = aa
        topology, positions, system, system_generator = generate_old_top_pos_sys(old_res_name)
        old_res, new_res = generate_inputs(new_res_name, topology, system, system_generator)
        current_oemol, proposed_oemol, atom_mapping = construct_atom_map(old_res, new_res, map_strength=map_strength)
        render_image(current_oemol, proposed_oemol, atom_mapping, filename=f"{old_res_name.upper()}_{new_res_name}_strength{map_strength}_map{i}.png")


DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


GLU->ARG Number of maps: 2
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (15, 15), (18, 18), (20, 20), (7, 7), (9, 9), (12, 11), (11, 12), (17, 16), (16, 17)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('CG', 'CG'), ('CD', 'CD'), ('HD3', 'OE2'), ('H', 'H'), ('HA', 'HA'), ('HB3', 'HB2'), ('HB2', 'HB3'), ('HG3', 'HG2'), ('HG2', 'HG3')]
GLU->HIS Number of maps: 1
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (15, 15), (21, 18), (22, 20), (7, 7), (9, 9), (12, 11), (11, 12)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('CG', 'CG'), ('CD2', 'CD'), ('HD2', 'OE2'), ('H', 'H'), ('HA', 'HA'), ('HB3', 'HB2'), ('HB2', 'HB3')]
GLU->LYS Number of maps: 2
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (15, 15), (18, 18), (20, 20), (7, 7), (9, 9), (12, 11), (11, 12), (17, 16), (16, 17)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('CG', 'CG'), ('CD', 'CD'), ('HD3', 'OE2'), ('H', 'H'), ('HA', 'HA'), ('HB3', 'HB2'), ('HB2', 'HB3'), ('HG3'

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


GLU->ASP Number of maps: 2
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (15, 15), (7, 7), (9, 9), (11, 11), (12, 12), (17, 17)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('CG', 'CG'), ('H', 'H'), ('HA', 'HA'), ('HB2', 'HB2'), ('HB3', 'HB3'), ('OD2', 'HG3')]
GLU->SER Number of maps: 1
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (7, 7), (9, 9), (11, 11), (12, 12)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('H', 'H'), ('HA', 'HA'), ('HB2', 'HB2'), ('HB3', 'HB3')]
GLU->THR Number of maps: 6
[(6, 6), (8, 8), (11, 13), (12, 14), (10, 10), (14, 15), (7, 7), (9, 9), (13, 11), (16, 16), (17, 17)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('CG2', 'CG'), ('H', 'H'), ('HA', 'HA'), ('HB', 'HB2'), ('HG22', 'HG2'), ('HG23', 'HG3')]
GLU->ASN Number of maps: 1
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (15, 15), (7, 7), (9, 9), (11, 11), (12, 12)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('CG', 'CG'), 

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


...

GLU->CYS Number of maps: 1
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (7, 7), (9, 9), (11, 11), (12, 12)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('H', 'H'), ('HA', 'HA'), ('HB2', 'HB2'), ('HB3', 'HB3')]
GLU->GLY Number of maps: 2
[(6, 6), (8, 8), (9, 13), (10, 14), (7, 7), (12, 9)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('H', 'H'), ('HA3', 'HA')]


DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


GLU->ALA Number of maps: 3
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (7, 7), (9, 9), (15, 11), (11, 12)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('H', 'H'), ('HA', 'HA'), ('HB1', 'HB2'), ('HB2', 'HB3')]
GLU->VAL Number of maps: 12
[(6, 6), (8, 8), (11, 13), (12, 14), (10, 10), (18, 15), (7, 7), (9, 9), (13, 11), (19, 16), (21, 17)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('CG2', 'CG'), ('H', 'H'), ('HA', 'HA'), ('HB', 'HB2'), ('HG21', 'HG2'), ('HG23', 'HG3')]
GLU->ILE Number of maps: 8
[(6, 6), (8, 8), (11, 13), (12, 14), (10, 10), (14, 15), (7, 7), (9, 9), (13, 11), (15, 16), (16, 17)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('CG2', 'CG'), ('H', 'H'), ('HA', 'HA'), ('HB', 'HB2'), ('HG21', 'HG2'), ('HG22', 'HG3')]
GLU->LEU Number of maps: 2
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (15, 15), (7, 7), (9, 9), (11, 11), (12, 12), (16, 17)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), (

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


GLU->MET Number of maps: 1
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (15, 15), (7, 7), (9, 9), (11, 11), (12, 12), (16, 16), (17, 17)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('CG', 'CG'), ('H', 'H'), ('HA', 'HA'), ('HB2', 'HB2'), ('HB3', 'HB3'), ('HG2', 'HG2'), ('HG3', 'HG3')]
GLU->PHE Number of maps: 1
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (7, 7), (9, 9), (11, 11), (12, 12)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('H', 'H'), ('HA', 'HA'), ('HB2', 'HB2'), ('HB3', 'HB3')]
GLU->TYR Number of maps: 1
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (7, 7), (9, 9), (11, 11), (12, 12)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('H', 'H'), ('HA', 'HA'), ('HB2', 'HB2'), ('HB3', 'HB3')]
GLU->TRP Number of maps: 1
[(6, 6), (8, 8), (13, 13), (14, 14), (10, 10), (7, 7), (9, 9), (11, 11), (12, 12)]
[('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('H', 'H'), ('HA', 'HA'), ('HB2', 'HB2'), ('HB3', 

In [6]:
import matplotlib.image
import numpy as np

amino_acids = ['ARG', 'HIS', 'LYS', 'ASP', 'GLU', 
               'SER', 'THR', 'ASN', 'GLN', 'CYS', 'GLY', 
               'ALA', 'VAL', 'ILE', 'LEU', 'MET',  'PHE',  'TYR', 'TRP']
old_res_name = "glu".upper()
map = 0
amino_acids.remove(old_res_name)

for map_strength in range(4):
    rows = []
    for i in range(6):
        row = []
        for aa in amino_acids[i*3:i*3+3]:
            if aa != old_res_name:
                img = matplotlib.image.imread(f'{old_res_name}_{aa}_strength{map_strength}_map{map}.png')
                row.append(img)
        rows.append(np.hstack(row))

    new_image = np.vstack(rows)

    matplotlib.image.imsave(f'{old_res_name.lower()}_combined_strength{map_strength}.png', new_image)


DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:b'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 59 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:b'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 59 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:b'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 59 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:b'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 59 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:b'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 59 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImageP

DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:b'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 59 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:b'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 59 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:b'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 59 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:b'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 59 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:b'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 59 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImageP

In [28]:
# Example code for showing problematic chirality in perceive_chirality()
old_res_name = 'ASN'
new_res_name = 'CYS'
current_residue_pdb_filename = resource_filename('perses', os.path.join('data', 'amino_acid_templates', f"{old_res_name}.pdb"))
proposed_residue_pdb_filename = resource_filename('perses', os.path.join('data', 'amino_acid_templates', f"{new_res_name}.pdb"))
current_oemol = PointMutationEngine.generate_oemol_from_pdb_template(current_residue_pdb_filename)
proposed_oemol = PointMutationEngine.generate_oemol_from_pdb_template(proposed_residue_pdb_filename)
atoms = list(current_oemol.GetAtoms())
print(atoms[1].GetName(), atoms[1].IsChiral())
new_to_old_atom_map = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 8, 7: 9, 9: 10, 8: 11}
atom_mapping = AtomMapping(current_oemol, proposed_oemol, new_to_old_atom_map=new_to_old_atom_map)
atoms = list(atom_mapping.old_mol.to_openeye().GetAtoms())
print(atoms[1].GetName(), atoms[1].IsChiral())


CA True
CA False
